In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
# 設置數據生成器
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    brightness_range=[0.9, 1.1],
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    shear_range=0.1,
    validation_split=0.2    
)

# 訓練數據生成器
train_generator = train_datagen.flow_from_directory(
    r'E:\Codes\CV train\回收系統dataset\Garbage',  # 替換為您的資料路徑
    target_size=(224, 224),  
    batch_size=16,
    class_mode='categorical',
    subset='training'  # 指定為訓練集
)

# 驗證數據生成器
validation_generator = train_datagen.flow_from_directory(
    r'E:\Codes\CV train\回收系統dataset\Garbage',
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical',
    subset='validation'  # 指定為驗證集
)

Found 6465 images belonging to 4 classes.
Found 1614 images belonging to 4 classes.


In [4]:
# 測試層數
unfreeze_layers = [-1 ,-2, -3, -4, -5, -6, -7, -8, -9]

# 逐步測試不同的解凍層數
for unfreeze in unfreeze_layers:
    print(f"\n==== 測試解凍層數: {unfreeze} ====")
    
    # 加載預訓練模型
    base_model = MobileNetV2(
        alpha=0.35, 
        weights='imagenet', 
        include_top=False, 
        input_shape=(224, 224, 3)
    )

    # 添加自定義分類層
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.4)(x)
    x = Dense(64,activation='relu')(x)
    x = Dense(4, activation='softmax')(x) 
    model = Model(inputs=base_model.input, outputs=x)

    # 設定解凍層
    for layer in base_model.layers[:unfreeze]:  
        layer.trainable = False

    # 編譯模型
    model.compile(
        optimizer='Adam', 
        loss='categorical_crossentropy', 
        metrics=['accuracy']
    )

    # 訓練模型
    history = model.fit(
        train_generator,
        validation_data=validation_generator,
        epochs=10,
        steps_per_epoch=train_generator.samples // 16,
        validation_steps=validation_generator.samples // 16,
    )


==== 測試解凍層數: -1 ====
Epoch 1/10
404/404 [==============================] - 73s 174ms/step - loss: 0.3870 - accuracy: 0.8403 - val_loss: 0.3903 - val_accuracy: 0.8469
Epoch 2/10
404/404 [==============================] - 63s 155ms/step - loss: 0.2677 - accuracy: 0.8935 - val_loss: 0.3837 - val_accuracy: 0.8431
Epoch 3/10
404/404 [==============================] - 64s 158ms/step - loss: 0.2481 - accuracy: 0.9028 - val_loss: 0.3398 - val_accuracy: 0.8681
Epoch 4/10
404/404 [==============================] - 62s 154ms/step - loss: 0.2269 - accuracy: 0.9101 - val_loss: 0.3352 - val_accuracy: 0.8656
Epoch 5/10
404/404 [==============================] - 62s 153ms/step - loss: 0.2047 - accuracy: 0.9192 - val_loss: 0.3583 - val_accuracy: 0.8594
Epoch 6/10
404/404 [==============================] - 61s 151ms/step - loss: 0.1875 - accuracy: 0.9237 - val_loss: 0.3756 - val_accuracy: 0.8662
Epoch 7/10
404/404 [==============================] - 61s 152ms/step - loss: 0.1830 - accuracy: 0.9291 - val

In [5]:
# 測試層數
big_unfreeze_layers = [-10 ,-20, -30, -40, -50, -60, -70, -80, -90, -100, -110, -120, -130, -140, -150]

# 逐步測試不同的解凍層數
for unfreeze in big_unfreeze_layers:
    print(f"\n==== 測試解凍層數: {unfreeze} ====")
    
    # 加載預訓練模型
    base_model = MobileNetV2(
        alpha=0.35, 
        weights='imagenet', 
        include_top=False, 
        input_shape=(224, 224, 3)
    )

    # 添加自定義分類層
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.4)(x)
    x = Dense(64,activation='relu')(x)
    x = Dense(4, activation='softmax')(x) 
    model = Model(inputs=base_model.input, outputs=x)

    # 設定解凍層
    for layer in base_model.layers[:unfreeze]:  
        layer.trainable = False

    # 編譯模型
    model.compile(
        optimizer='Adam', 
        loss='categorical_crossentropy', 
        metrics=['accuracy']
    )

    # 訓練模型
    history = model.fit(
        train_generator,
        validation_data=validation_generator,
        epochs=10,
        steps_per_epoch=train_generator.samples // 16,
        validation_steps=validation_generator.samples // 16,
    )


==== 測試解凍層數: -10 ====
Epoch 1/10
404/404 [==============================] - 66s 161ms/step - loss: 0.3813 - accuracy: 0.8527 - val_loss: 3.4327 - val_accuracy: 0.5744
Epoch 2/10
404/404 [==============================] - 64s 160ms/step - loss: 0.2710 - accuracy: 0.8930 - val_loss: 4.3281 - val_accuracy: 0.5331
Epoch 3/10
404/404 [==============================] - 65s 161ms/step - loss: 0.2323 - accuracy: 0.9125 - val_loss: 0.8051 - val_accuracy: 0.7987
Epoch 4/10
404/404 [==============================] - 64s 159ms/step - loss: 0.2084 - accuracy: 0.9223 - val_loss: 0.7211 - val_accuracy: 0.8263
Epoch 5/10
404/404 [==============================] - 65s 160ms/step - loss: 0.2058 - accuracy: 0.9198 - val_loss: 1.2165 - val_accuracy: 0.7700
Epoch 6/10
404/404 [==============================] - 65s 160ms/step - loss: 0.1917 - accuracy: 0.9304 - val_loss: 0.9984 - val_accuracy: 0.7837
Epoch 7/10
404/404 [==============================] - 65s 161ms/step - loss: 0.1619 - accuracy: 0.9386 - va